# Notebook 02: Funciones y Modularidad

---

## Objetivos de aprendizaje

Al finalizar este notebook seras capaz de:

1. **Definir funciones** con type hints, docstrings y parametros por defecto.
2. **Entender scope y closures** aplicando la regla LEGB.
3. **Detectar el anti-patron** de argumentos mutables por defecto y corregirlo.
4. **Organizar codigo en modulos** con una estructura de proyecto limpia.
5. **Aplicar buenas practicas** (DRY, responsabilidad unica, naming conventions).

| Temas cubiertos | Referencia |
|---|---|
| Funciones y type hints | `03_funciones` |
| Modulos y archivos | `06_modulos_y_archivos` |
| Buenas practicas | `07_buenas_practicas` |

**Tiempo estimado:** 60-90 minutos

**Prerequisito:** Notebook 01 (variables, control de flujo, estructuras de datos)

---

## 1. Anatomia de una Funcion

Cada funcion en Python sigue una estructura precisa. En ingenieria de ML, las funciones
son los bloques fundamentales de cualquier pipeline.

```
    +-- keyword          +-- nombre          +-- parametros con type hints
    |                    |                    |
    v                    v                    v
   def  calculate_accuracy(correct: int, total: int) -> float:
        """                                                      <-- docstring
        Calcula la accuracy de un modelo.                        
        """                                                      
        if total == 0:                                           <-- cuerpo
            return 0.0                                           
        return correct / total                                   <-- return
             ^                    ^
             |                    |
             +-- valor de retorno +-- tipo de retorno: float
```

### Tipos de parametros

| Tipo | Sintaxis | Ejemplo | Uso en ML |
|---|---|---|---|
| Posicional | `param` | `def f(x)` | Datos de entrada obligatorios |
| Con valor por defecto | `param=valor` | `def f(lr=0.01)` | Hiperparametros |
| Keyword-only | `*, param` | `def f(*, verbose=True)` | Flags de configuracion |
| `*args` | `*args` | `def f(*layers)` | Capas variables en una red |
| `**kwargs` | `**kwargs` | `def f(**config)` | Configuracion flexible |

In [ ]:
# --- Definicion basica con type hints y docstrings ---

def calculate_accuracy(correct: int, total: int) -> float:
    """Calcula la accuracy dado el numero de aciertos y el total.
    
    Args:
        correct: Numero de predicciones correctas.
        total: Numero total de predicciones.
    
    Returns:
        Accuracy como un float entre 0.0 y 1.0.
    """
    if total == 0:
        return 0.0
    return correct / total


def classify_score(score: float, threshold: float = 0.5) -> str:
    """Clasifica un score de modelo como POSITIVO o NEGATIVO.
    
    Args:
        score: Probabilidad predicha por el modelo (0.0 a 1.0).
        threshold: Umbral de decision (por defecto 0.5).
    
    Returns:
        'POSITIVO' si score >= threshold, 'NEGATIVO' en caso contrario.
    """
    return "POSITIVO" if score >= threshold else "NEGATIVO"


# --- Uso ---
acc = calculate_accuracy(correct=85, total=100)
print(f"Accuracy del modelo: {acc:.2%}")

# Con threshold por defecto (0.5)
print(f"Score 0.73 -> {classify_score(0.73)}")
print(f"Score 0.32 -> {classify_score(0.32)}")

# Con threshold personalizado
print(f"Score 0.73 (umbral=0.8) -> {classify_score(0.73, threshold=0.8)}")

# Podemos inspeccionar los type hints y el docstring
print(f"\nType hints: {calculate_accuracy.__annotations__}")
print(f"Docstring:\n{calculate_accuracy.__doc__}")

## 2. Scope: La Regla LEGB

Python busca variables en cuatro niveles, de adentro hacia afuera:

```
  +-----------------------------------------------------------+
  |  Built-in (B)                                             |
  |  print(), len(), range(), int(), str(), ...               |
  |                                                           |
  |  +-----------------------------------------------------+ |
  |  |  Global (G)                                          | |
  |  |  Variables definidas a nivel de modulo               | |
  |  |  MODEL_VERSION = "2.1"                               | |
  |  |                                                      | |
  |  |  +------------------------------------------------+ | |
  |  |  |  Enclosing (E)                                  | | |
  |  |  |  Variables de la funcion externa (closures)     | | |
  |  |  |  threshold = 0.5                                | | |
  |  |  |                                                 | | |
  |  |  |  +-------------------------------------------+ | | |
  |  |  |  |  Local (L)                                 | | | |
  |  |  |  |  Variables dentro de la funcion actual     | | | |
  |  |  |  |  result = score > threshold                | | | |
  |  |  |  +-------------------------------------------+ | | |
  |  |  +------------------------------------------------+ | |
  |  +-----------------------------------------------------+ |
  +-----------------------------------------------------------+

  Orden de busqueda:  L -> E -> G -> B
```

**Regla clave:** Python siempre busca desde el scope mas interno hacia el mas externo.
Si no encuentra la variable en ninguno, lanza `NameError`.

In [ ]:
# --- Demostracion de la regla LEGB ---

# Global (G)
MODEL_NAME = "ClassifierV1"

def show_scope():
    # Local (L) - esta variable "sombrea" cualquier variable global con el mismo nombre
    model_info = "informacion local"
    print(f"  Local -> model_info = '{model_info}'")
    print(f"  Global -> MODEL_NAME = '{MODEL_NAME}'")
    print(f"  Built-in -> len = {len}")

print("--- Scope basico ---")
show_scope()

# --- Closures: Enclosing scope ---
print("\n--- Closures ---")

def create_threshold_classifier(threshold: float):
    """Retorna una funcion que clasifica usando el threshold capturado.
    
    Esto es un closure: la funcion interna 'recuerda' el valor de threshold
    incluso despues de que create_threshold_classifier haya terminado.
    """
    print(f"  Creando clasificador con threshold={threshold}")
    
    def classify(score: float) -> str:
        # 'threshold' viene del Enclosing scope (E)
        return "POSITIVO" if score >= threshold else "NEGATIVO"
    
    return classify

# Creamos dos clasificadores con distintos umbrales
clf_conservador = create_threshold_classifier(0.8)
clf_agresivo = create_threshold_classifier(0.3)

score_prueba = 0.65
print(f"\n  Score: {score_prueba}")
print(f"  Clasificador conservador (0.8): {clf_conservador(score_prueba)}")
print(f"  Clasificador agresivo    (0.3): {clf_agresivo(score_prueba)}")

# Verificamos que el closure captura la variable
print(f"\n  Variables capturadas por clf_conservador: {clf_conservador.__closure__[0].cell_contents}")

## 3. Anti-patron: Argumentos Mutables por Defecto

> **ADVERTENCIA:** Este es uno de los errores mas comunes y dificiles de detectar
> en Python. Los valores por defecto mutables (`list`, `dict`, `set`) se crean
> **una sola vez** cuando se define la funcion, NO cada vez que se llama.

```
  PELIGROSO                          CORRECTO
  +-----------------------+          +---------------------------+
  | def f(x, lst=[]):     |          | def f(x, lst=None):       |
  |     lst.append(x)     |          |     if lst is None:       |
  |     return lst        |          |         lst = []          |
  +-----------------------+          |     lst.append(x)         |
  La misma lista se reutiliza        |     return lst            |
  en cada llamada!                   +---------------------------+
                                     Lista nueva en cada llamada
```

In [ ]:
# --- El bug de argumentos mutables por defecto ---

print("=" * 55)
print("VERSION CON BUG: default mutable argument")
print("=" * 55)

def append_to_buggy(element, target=[]):
    """BUGGY: la lista target se comparte entre llamadas."""
    target.append(element)
    return target

# Cada llamada deberia devolver una lista con UN solo elemento...
result1 = append_to_buggy("modelo_A")
print(f"Llamada 1: {result1}")        # Esperado: ['modelo_A']

result2 = append_to_buggy("modelo_B")
print(f"Llamada 2: {result2}")        # Esperado: ['modelo_B'], Real: ['modelo_A', 'modelo_B'] !!

result3 = append_to_buggy("modelo_C")
print(f"Llamada 3: {result3}")        # La lista sigue creciendo!

print(f"\nresult1 is result2: {result1 is result2}")  # Son el MISMO objeto!


print("\n" + "=" * 55)
print("VERSION CORREGIDA: None como default")
print("=" * 55)

def append_to_fixed(element, target=None):
    """CORRECTO: se crea una lista nueva si no se proporciona una."""
    if target is None:
        target = []
    target.append(element)
    return target

result1 = append_to_fixed("modelo_A")
print(f"Llamada 1: {result1}")        # ['modelo_A']

result2 = append_to_fixed("modelo_B")
print(f"Llamada 2: {result2}")        # ['modelo_B'] -- correcto!

result3 = append_to_fixed("modelo_C")
print(f"Llamada 3: {result3}")        # ['modelo_C'] -- correcto!

print(f"\nresult1 is result2: {result1 is result2}")  # Son objetos DIFERENTES

## 4. Funciones como Objetos de Primera Clase

En Python, las funciones son **objetos**. Esto significa que puedes:

- Asignarlas a variables
- Pasarlas como argumentos a otras funciones (**higher-order functions**)
- Retornarlas desde otras funciones (ya lo vimos con closures)

Esto es fundamental para construir **pipelines de procesamiento** en ML:

```
  datos_crudos --> [transformar] --> [filtrar] --> [ordenar] --> resultado
                       |                |             |
                    map(fn)        filter(fn)    sorted(key=fn)
```

### `lambda` vs funcion nombrada

| Aspecto | `lambda` | Funcion nombrada |
|---|---|---|
| Sintaxis | `lambda x: x * 2` | `def double(x): return x * 2` |
| Lineas | Solo una expresion | Multiples lineas |
| Legibilidad | Menor (logica simple) | Mayor (logica compleja) |
| Depuracion | Sin nombre en traceback | Nombre claro en traceback |
| Uso ideal | `sorted(key=lambda...)` | Reutilizacion, testing |

In [ ]:
# --- Funciones de orden superior con contexto ML ---

# Predicciones de un modelo (probabilidades)
predictions = [0.12, 0.89, 0.45, 0.67, 0.93, 0.31, 0.78, 0.55, 0.08, 0.72]

# --- map(): aplicar transformacion a cada elemento ---
print("--- map(): Redondear predicciones ---")
rounded = list(map(lambda p: round(p, 1), predictions))
print(f"  Original:   {predictions}")
print(f"  Redondeado: {rounded}")

# --- filter(): quedarnos con elementos que cumplen condicion ---
print("\n--- filter(): Predicciones de alta confianza (>= 0.7) ---")
high_confidence = list(filter(lambda p: p >= 0.7, predictions))
print(f"  Alta confianza: {high_confidence}")

# --- sorted() con key: ordenar por criterio personalizado ---
print("\n--- sorted(): Ordenar por distancia al umbral 0.5 ---")

def distance_to_threshold(score: float, threshold: float = 0.5) -> float:
    """Calcula que tan lejos esta un score del umbral de decision."""
    return abs(score - threshold)

sorted_by_uncertainty = sorted(predictions, key=lambda p: distance_to_threshold(p))
print(f"  Mas cercanos al umbral (mas inciertos) primero:")
print(f"  {sorted_by_uncertainty}")

# --- Pipeline de transformaciones ---
print("\n--- Pipeline completo ---")
print(f"  Datos crudos:     {predictions}")

# Paso 1: normalizar al rango [0, 100]
step1 = list(map(lambda p: p * 100, predictions))
print(f"  Paso 1 (escalar): {step1}")

# Paso 2: filtrar scores >= 50
step2 = list(filter(lambda s: s >= 50, step1))
print(f"  Paso 2 (filtrar): {step2}")

# Paso 3: ordenar de mayor a menor
step3 = sorted(step2, reverse=True)
print(f"  Paso 3 (ordenar): {step3}")

# Paso 4: formatear como porcentajes
step4 = list(map(lambda s: f"{s:.0f}%", step3))
print(f"  Resultado final:  {step4}")

## 5. Organizacion en Modulos

Un proyecto de ML bien organizado separa responsabilidades en modulos:

```
  mi_proyecto_ml/
  |
  +-- main.py                  # Punto de entrada
  +-- config.py                # Constantes y configuracion
  |
  +-- data/
  |   +-- __init__.py
  |   +-- loader.py            # Funciones de carga de datos
  |   +-- preprocessing.py     # Limpieza y transformaciones
  |
  +-- models/
  |   +-- __init__.py
  |   +-- classifier.py        # Definicion del modelo
  |   +-- metrics.py           # Funciones de evaluacion
  |
  +-- utils/
  |   +-- __init__.py
  |   +-- file_io.py           # Lectura/escritura de archivos
  |   +-- logging_utils.py     # Utilidades de logging
  |
  +-- tests/
      +-- test_metrics.py
      +-- test_preprocessing.py
```

### Conceptos clave

| Concepto | Que es | Para que sirve |
|---|---|---|
| Modulo | Un archivo `.py` | Agrupar funciones relacionadas |
| Paquete | Carpeta con `__init__.py` | Agrupar modulos relacionados |
| `import` | Cargar un modulo | Reutilizar codigo |
| `__name__` | Nombre del modulo actual | Distinguir ejecucion directa vs importacion |
| `if __name__ == "__main__":` | Guardia de ejecucion | Ejecutar solo si se corre directamente |

In [ ]:
# --- Simulacion de organizacion modular ---
# En un notebook no podemos crear archivos .py separados facilmente,
# pero podemos demostrar los conceptos.

# --- Simulacion de config.py ---
# En un modulo real, estas serian constantes a nivel de modulo
LEARNING_RATE = 0.001
BATCH_SIZE = 32
MODEL_VERSION = "1.0.0"

# --- Simulacion de metrics.py ---
def compute_precision(true_positives: int, false_positives: int) -> float:
    """Calcula precision: TP / (TP + FP)."""
    total = true_positives + false_positives
    return true_positives / total if total > 0 else 0.0

def compute_recall(true_positives: int, false_negatives: int) -> float:
    """Calcula recall: TP / (TP + FN)."""
    total = true_positives + false_negatives
    return true_positives / total if total > 0 else 0.0

def compute_f1(precision: float, recall: float) -> float:
    """Calcula F1 score: 2 * (P * R) / (P + R)."""
    total = precision + recall
    return 2 * (precision * recall) / total if total > 0 else 0.0

# --- Simulacion de main.py ---
def main():
    """Funcion principal del pipeline de evaluacion."""
    print(f"Modelo: {MODEL_VERSION}")
    print(f"Config: lr={LEARNING_RATE}, batch_size={BATCH_SIZE}")
    print()
    
    # Resultados de prediccion simulados
    tp, fp, fn = 80, 15, 20
    
    prec = compute_precision(tp, fp)
    rec = compute_recall(tp, fn)
    f1 = compute_f1(prec, rec)
    
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1 Score:  {f1:.4f}")

# --- El patron if __name__ == "__main__" ---
# En un notebook, __name__ siempre es "__main__"
# En un modulo importado, __name__ seria el nombre del modulo
print(f"Valor de __name__ en este notebook: '{__name__}'")
print()

if __name__ == "__main__":
    # Este bloque SOLO se ejecuta si corremos el script directamente.
    # Si otro modulo hace 'import este_modulo', este bloque NO se ejecuta.
    main()

## 6. File I/O: Leer y Escribir Datos

En un proyecto real de ML, constantemente leemos y escribimos archivos:
configuraciones, datasets, logs de entrenamiento, resultados de metricas.

### Modos de apertura

| Modo | Descripcion | Ejemplo de uso |
|---|---|---|
| `'r'` | Lectura (texto) | Leer configuracion, datasets CSV |
| `'w'` | Escritura (sobreescribe) | Guardar resultados, logs |
| `'a'` | Append (agrega al final) | Agregar entradas a un log |
| `'rb'` | Lectura binaria | Cargar modelo serializado |
| `'wb'` | Escritura binaria | Guardar modelo serializado |

### Context managers (`with`)

```python
# SIEMPRE usar 'with' para manejo automatico de recursos:
with open("archivo.txt", "r") as f:
    contenido = f.read()
# El archivo se cierra automaticamente al salir del bloque
```

> **Regla:** Nunca uses `f = open(...)` sin `with`. Si ocurre un error,
> el archivo podria quedar abierto y causar problemas.

In [ ]:
import tempfile
import os

# Usamos tempfile para no dejar archivos basura en el sistema
temp_dir = tempfile.mkdtemp()

# --- Escritura: guardar resultados de experimentos ---
experiment_file = os.path.join(temp_dir, "experiment_results.csv")

experiment_data = [
    {"epoch": 1, "loss": 2.341, "accuracy": 0.45},
    {"epoch": 2, "loss": 1.892, "accuracy": 0.58},
    {"epoch": 3, "loss": 1.234, "accuracy": 0.71},
    {"epoch": 4, "loss": 0.876, "accuracy": 0.79},
    {"epoch": 5, "loss": 0.654, "accuracy": 0.84},
]

print("--- Escribiendo datos ---")
with open(experiment_file, "w") as f:
    # Header
    f.write("epoch,loss,accuracy\n")
    # Datos
    for record in experiment_data:
        line = f"{record['epoch']},{record['loss']:.3f},{record['accuracy']:.2f}\n"
        f.write(line)

print(f"  Archivo escrito en: {experiment_file}")

# --- Lectura: cargar y parsear los datos ---
print("\n--- Leyendo datos ---")
parsed_records = []

with open(experiment_file, "r") as f:
    header = f.readline().strip().split(",")
    print(f"  Columnas: {header}")
    
    for line in f:
        parts = line.strip().split(",")
        record = {
            "epoch": int(parts[0]),
            "loss": float(parts[1]),
            "accuracy": float(parts[2]),
        }
        parsed_records.append(record)

# Mostrar los datos parseados
print(f"  Registros cargados: {len(parsed_records)}")
for rec in parsed_records:
    status = "MEJORANDO" if rec["accuracy"] > 0.7 else "entrenando..."
    print(f"  Epoch {rec['epoch']:2d} | Loss: {rec['loss']:.3f} | Acc: {rec['accuracy']:.2f} | {status}")

# --- Append: agregar un nuevo resultado ---
print("\n--- Agregando nuevo registro (append) ---")
with open(experiment_file, "a") as f:
    f.write("6,0.512,0.87\n")

# Verificar que se agrego
with open(experiment_file, "r") as f:
    all_lines = f.readlines()
    print(f"  Total lineas (con header): {len(all_lines)}")
    print(f"  Ultima linea: {all_lines[-1].strip()}")

# Limpiar archivos temporales
os.remove(experiment_file)
os.rmdir(temp_dir)
print(f"\n  Archivos temporales limpiados.")

## 7. Buenas Practicas

Escribir codigo que funciona no es suficiente. En equipos de ingenieria de ML,
el codigo debe ser **legible, mantenible y testeable**.

### Convenciones de naming

| Elemento | Convencion | Ejemplo |
|---|---|---|
| Funciones | `snake_case` | `calculate_accuracy()` |
| Variables | `snake_case` | `learning_rate` |
| Constantes | `UPPER_SNAKE_CASE` | `MAX_EPOCHS = 100` |
| Clases | `PascalCase` | `DataLoader` |
| Modulos | `snake_case` | `data_utils.py` |
| Variables privadas | `_prefijo` | `_internal_cache` |

### Principios fundamentales

| Principio | Descripcion | Ejemplo |
|---|---|---|
| **DRY** | Don't Repeat Yourself | Extraer logica duplicada a una funcion |
| **SRP** | Single Responsibility Principle | Cada funcion hace UNA cosa |
| **KISS** | Keep It Simple, Stupid | Preferir claridad sobre "elegancia" |
| **Funciones pequenas** | Max 20-30 lineas | Si es mas larga, dividirla |
| **Docstrings** | Documentar el "por que" | No el "que" (el codigo ya dice eso) |

In [ ]:
# --- COMPARACION: Codigo malo vs codigo bueno ---

# ============================================================
# VERSION MALA: Una funcion monolitica que hace todo
# ============================================================
print("=" * 60)
print("VERSION MALA: Funcion monolitica")
print("=" * 60)

def process_everything(raw_data):
    """Hace DEMASIADAS cosas en una sola funcion."""
    # Validar
    clean = []
    for d in raw_data:
        if d is not None and isinstance(d, dict) and "score" in d and "name" in d:
            clean.append(d)
    # Transformar
    for d in clean:
        d["score_pct"] = round(d["score"] * 100, 1)
        d["label"] = "APROBADO" if d["score"] >= 0.6 else "REPROBADO"
    # Calcular resumen
    total = len(clean)
    aprobados = len([d for d in clean if d["label"] == "APROBADO"])
    promedio = sum(d["score"] for d in clean) / total if total > 0 else 0
    # Imprimir
    for d in clean:
        print(f"  {d['name']}: {d['score_pct']}% -> {d['label']}")
    print(f"  --- Resumen: {aprobados}/{total} aprobados, promedio={promedio:.2f}")
    return clean

datos_crudos = [
    {"name": "modelo_A", "score": 0.85},
    None,
    {"name": "modelo_B", "score": 0.42},
    {"score": 0.9},  # falta 'name'
    {"name": "modelo_C", "score": 0.73},
    {"name": "modelo_D", "score": 0.55},
]

process_everything(datos_crudos)


# ============================================================
# VERSION BUENA: Funciones pequenas con responsabilidad unica
# ============================================================
print("\n" + "=" * 60)
print("VERSION BUENA: Funciones con responsabilidad unica")
print("=" * 60)

PASSING_THRESHOLD = 0.6

def validate_record(record) -> bool:
    """Verifica que un registro tenga los campos requeridos."""
    if record is None or not isinstance(record, dict):
        return False
    required_fields = {"name", "score"}
    return required_fields.issubset(record.keys())


def enrich_record(record: dict) -> dict:
    """Agrega campos calculados a un registro."""
    return {
        **record,
        "score_pct": round(record["score"] * 100, 1),
        "label": "APROBADO" if record["score"] >= PASSING_THRESHOLD else "REPROBADO",
    }


def summarize(records: list) -> dict:
    """Calcula estadisticas de resumen."""
    total = len(records)
    aprobados = sum(1 for r in records if r["label"] == "APROBADO")
    promedio = sum(r["score"] for r in records) / total if total > 0 else 0.0
    return {"total": total, "aprobados": aprobados, "promedio": promedio}


def display_results(records: list, summary: dict) -> None:
    """Muestra los resultados formateados."""
    for r in records:
        print(f"  {r['name']}: {r['score_pct']}% -> {r['label']}")
    print(f"  --- Resumen: {summary['aprobados']}/{summary['total']} aprobados, "
          f"promedio={summary['promedio']:.2f}")


# Pipeline claro y legible
valid = [r for r in datos_crudos if validate_record(r)]
enriched = [enrich_record(r) for r in valid]
summary = summarize(enriched)
display_results(enriched, summary)

print("\n[Ventajas: cada funcion se puede testear, reutilizar y entender por separado]")

---

## Ejercicios

Pon en practica lo aprendido. Cada ejercicio incluye instrucciones detalladas.
Completa las funciones reemplazando `pass` con tu implementacion.

---

In [ ]:
# ============================================================
# EJERCICIO 1: Closure - Normalizador configurable
# ============================================================
# Crea una funcion make_normalizer(min_val, max_val) que retorne
# una funcion (closure) capaz de normalizar cualquier valor al
# rango [0, 1] usando la formula:
#
#   normalizado = (valor - min_val) / (max_val - min_val)
#
# Ejemplo de uso esperado:
#   normalizer = make_normalizer(0, 100)
#   normalizer(50)   -> 0.5
#   normalizer(0)    -> 0.0
#   normalizer(100)  -> 1.0
#
# BONUS: Que pasa si min_val == max_val? Maneja ese caso.
# ============================================================

def make_normalizer(min_val: float, max_val: float):
    """Retorna una funcion que normaliza valores al rango [0, 1]."""
    pass


# --- Tests (descomenta para verificar tu solucion) ---
# normalizer = make_normalizer(0, 100)
# assert normalizer(50) == 0.5
# assert normalizer(0) == 0.0
# assert normalizer(100) == 1.0
# print("Ejercicio 1: OK")

In [ ]:
# ============================================================
# EJERCICIO 2: Pipeline de procesamiento con funciones pequenas
# ============================================================
# Dado una lista de registros de experimentos (diccionarios),
# implementa 3 funciones con responsabilidad unica:
#
# 1. validate_experiment(record) -> bool
#    Verifica que el registro tenga las claves: "name", "accuracy", "loss"
#    y que accuracy este entre 0 y 1, y loss sea >= 0.
#
# 2. transform_experiment(record) -> dict
#    Agrega el campo "status":
#    - "excelente" si accuracy >= 0.9
#    - "bueno" si accuracy >= 0.7
#    - "necesita_mejora" en otro caso
#
# 3. summarize_experiments(records) -> dict
#    Retorna {"total", "mejor_modelo" (nombre con mayor accuracy),
#             "accuracy_promedio"}
#
# Luego aplica las 3 funciones en secuencia sobre los datos de prueba.
# ============================================================

# Datos de prueba
raw_experiments = [
    {"name": "bert_base", "accuracy": 0.92, "loss": 0.21},
    {"name": "lstm_v1", "accuracy": 0.75, "loss": 0.68},
    {"name": "bad_record"},  # invalido
    {"name": "transformer_xl", "accuracy": 0.88, "loss": 0.31},
    {"name": "logistic_reg", "accuracy": 0.65, "loss": 0.89},
    {"name": "invalid", "accuracy": 1.5, "loss": -0.1},  # invalido
]


def validate_experiment(record: dict) -> bool:
    """Valida que un registro de experimento tenga campos correctos."""
    pass


def transform_experiment(record: dict) -> dict:
    """Agrega el campo 'status' basado en la accuracy."""
    pass


def summarize_experiments(records: list) -> dict:
    """Calcula resumen: total, mejor modelo y accuracy promedio."""
    pass


# --- Aplica tu pipeline aqui ---
# valid = ...
# transformed = ...
# summary = ...
# print(summary)

In [ ]:
# ============================================================
# EJERCICIO 3: File I/O con context managers
# ============================================================
# Implementa 3 funciones para un flujo de lectura-proceso-escritura:
#
# 1. write_training_log(filepath, records)
#    Escribe una lista de dicts [{"epoch": 1, "loss": 0.5, "acc": 0.8}, ...]
#    como CSV con header. Usa context manager.
#
# 2. read_training_log(filepath) -> list[dict]
#    Lee el CSV y retorna una lista de dicts con los tipos correctos
#    (epoch como int, loss y acc como float). Usa context manager.
#
# 3. generate_report(records) -> str
#    Genera un string con resumen: mejor epoch, peor loss, tendencia
#    (mejorando/empeorando basado en si el ultimo loss < primer loss).
#
# Usa tempfile para las operaciones de archivo.
# ============================================================

import tempfile
import os


def write_training_log(filepath: str, records: list) -> None:
    """Escribe registros de entrenamiento como CSV."""
    pass


def read_training_log(filepath: str) -> list:
    """Lee un CSV de entrenamiento y retorna lista de dicts."""
    pass


def generate_report(records: list) -> str:
    """Genera un reporte de resumen del entrenamiento."""
    pass


# --- Datos de prueba ---
# training_data = [
#     {"epoch": 1, "loss": 2.50, "acc": 0.35},
#     {"epoch": 2, "loss": 1.80, "acc": 0.52},
#     {"epoch": 3, "loss": 1.20, "acc": 0.68},
#     {"epoch": 4, "loss": 0.75, "acc": 0.81},
#     {"epoch": 5, "loss": 0.50, "acc": 0.89},
# ]
#
# temp_path = os.path.join(tempfile.mkdtemp(), "training_log.csv")
# write_training_log(temp_path, training_data)
# loaded = read_training_log(temp_path)
# report = generate_report(loaded)
# print(report)
# os.remove(temp_path)

---

## Checklist de consolidacion

Antes de avanzar al Notebook 03, verifica que puedes:

- [ ] Definir funciones con **type hints**, **docstrings** y **parametros por defecto**
- [ ] Explicar la regla **LEGB** y predecir donde Python busca una variable
- [ ] Crear y usar **closures** para encapsular configuracion
- [ ] Identificar y corregir el **anti-patron de argumentos mutables por defecto**
- [ ] Usar `map()`, `filter()`, `sorted()` con funciones como argumento
- [ ] Estructurar un proyecto con **modulos** y el patron `if __name__ == "__main__"`
- [ ] Leer y escribir archivos usando **context managers** (`with open(...)`)
- [ ] Aplicar **SRP** (Single Responsibility Principle) para refactorizar funciones largas

### Siguiente paso

**Notebook 03** cubrira: Programacion Orientada a Objetos, clases, herencia y patrones
de diseno aplicados a pipelines de ML.

---

*Curso de AI Engineering - Henry 2026*